In [1]:
from matplotlib.colors import ListedColormap
from sklearn.preprocessing import StandardScaler
from sklearn.datasets import make_moons, make_circles, make_classification
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.naive_bayes import GaussianNB
from keras.utils import np_utils
from keras.models import Sequential
from keras.layers import Merge, TimeDistributed, LSTM
from keras.layers.core import Dense, Dropout, Activation, Flatten, Reshape
from keras.layers.convolutional import Convolution1D, MaxPooling1D, Convolution2D, MaxPooling2D, ZeroPadding1D, Convolution3D, MaxPooling3D, AveragePooling3D
from keras.optimizers import SGD, Adadelta
from keras.callbacks import EarlyStopping
from keras.layers import Input, merge
from keras.models import Model
from keras import backend as K
from sklearn.cross_validation import train_test_split
import pandas as pd
import numpy as np

Using TensorFlow backend.
C:\Program Files\Anaconda3\envs\venv\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
data = pd.read_csv('E://csv_data/half/merge.csv', header = None)

In [3]:
data = data.reindex(np.random.permutation(data.index))
train, test = train_test_split(data, train_size=0.9, random_state=0)
x_train, x_test = train.iloc[:,1:].values, test.iloc[:,1:].values
y_train, y_test = np_utils.to_categorical(train.iloc[:,0].values), np_utils.to_categorical(test.iloc[:,0].values)

In [4]:
input_shape = (25600, )
output_shape = 2

In [8]:
# 3D-CNN Model (Except Attention)
model = Sequential()
model.add(Reshape(input_shape=input_shape, target_shape=(8, 8, 8, 50)))


model.add(Convolution3D(64, 3, 3, 3, border_mode='valid'))
model.add(Activation('tanh'))
model.add(Convolution3D(64, 3, 3, 3, border_mode='valid'))
model.add(Activation('tanh'))
model.add(Convolution3D(64, 3, 3, 3, border_mode='valid'))
model.add(Activation('tanh'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Activation('tanh'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(output_shape))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_2 (Reshape)          (None, 8, 8, 8, 50)       0         
_________________________________________________________________
conv3d_4 (Conv3D)            (None, 6, 6, 6, 64)       86464     
_________________________________________________________________
activation_7 (Activation)    (None, 6, 6, 6, 64)       0         
_________________________________________________________________
conv3d_5 (Conv3D)            (None, 4, 4, 4, 64)       110656    
_________________________________________________________________
activation_8 (Activation)    (None, 4, 4, 4, 64)       0         
_________________________________________________________________
conv3d_6 (Conv3D)            (None, 2, 2, 2, 64)       110656    
_________________________________________________________________
activation_9 (Activation)    (None, 2, 2, 2, 64)       0         
__________

C:\Program Files\Anaconda3\envs\venv\lib\site-packages\ipykernel\__main__.py:17: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), padding="valid")`
C:\Program Files\Anaconda3\envs\venv\lib\site-packages\ipykernel\__main__.py:19: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), padding="valid")`
C:\Program Files\Anaconda3\envs\venv\lib\site-packages\ipykernel\__main__.py:21: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), padding="valid")`


In [9]:
adadelta = Adadelta(lr=1.5, decay=1e-6)
model.compile(optimizer=adadelta,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [10]:
# 3D-CNN Model (Except Attention)
history = model.fit(x_train, y_train,
                    batch_size=100,
                    epochs=500,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 26280 samples, validate on 2921 samples
Epoch 1/500
26280/26280 [==============================] - 43s - loss: 0.7449 - acc: 0.5688 - val_loss: 0.6802 - val_acc: 0.6176
Epoch 2/500
26280/26280 [==============================] - 42s - loss: 0.6971 - acc: 0.5883 - val_loss: 0.6553 - val_acc: 0.6224
Epoch 3/500
26280/26280 [==============================] - 42s - loss: 0.6752 - acc: 0.6061 - val_loss: 0.6208 - val_acc: 0.6220
Epoch 4/500
26280/26280 [==============================] - 42s - loss: 0.6454 - acc: 0.6331 - val_loss: 0.5849 - val_acc: 0.7052
Epoch 5/500
26280/26280 [==============================] - 42s - loss: 0.6109 - acc: 0.6772 - val_loss: 0.5381 - val_acc: 0.7470
Epoch 6/500
26280/26280 [==============================] - 42s - loss: 0.5869 - acc: 0.6978 - val_loss: 0.5381 - val_acc: 0.7689
Epoch 7/500
26280/26280 [==============================] - 44s - loss: 0.5638 - acc: 0.7189 - val_loss: 0.5823 - val_acc: 0.7008
Epoch 8/500
26280/26280 [=======================

In [11]:
# 2D-CNN Model (Except Attention)
model = Sequential()
model.add(Reshape(input_shape=input_shape, target_shape=(64, 8, 50)))

model.add(Convolution2D(64, 3, 3, border_mode='valid'))
model.add(Activation('tanh'))
model.add(Convolution2D(64, 3, 3, border_mode='valid'))
model.add(Activation('tanh'))
model.add(Convolution2D(64, 3, 3, border_mode='valid'))
model.add(Activation('tanh'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('tanh'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(output_shape))
model.add(Activation('softmax'))

model.summary()

C:\Program Files\Anaconda3\envs\venv\lib\site-packages\ipykernel\__main__.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="valid")`
C:\Program Files\Anaconda3\envs\venv\lib\site-packages\ipykernel\__main__.py:7: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="valid")`
C:\Program Files\Anaconda3\envs\venv\lib\site-packages\ipykernel\__main__.py:9: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="valid")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_3 (Reshape)          (None, 64, 8, 50)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 62, 6, 64)         28864     
_________________________________________________________________
activation_13 (Activation)   (None, 62, 6, 64)         0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 60, 4, 64)         36928     
_________________________________________________________________
activation_14 (Activation)   (None, 60, 4, 64)         0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 58, 2, 64)         36928     
_________________________________________________________________
activation_15 (Activation)   (None, 58, 2, 64)         0         
__________

In [12]:
adadelta = Adadelta(lr=1.5, decay=1e-6)
model.compile(optimizer=adadelta,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [13]:
# 2D-CNN Model (Except Attention)
history = model.fit(x_train, y_train,
                    batch_size=100,
                    epochs=500,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 26280 samples, validate on 2921 samples
Epoch 1/500
26280/26280 [==============================] - 48s - loss: 0.7338 - acc: 0.6852 - val_loss: 0.2908 - val_acc: 0.9017
Epoch 2/500
26280/26280 [==============================] - 39s - loss: 0.3434 - acc: 0.8555 - val_loss: 0.1363 - val_acc: 0.9651
Epoch 3/500
26280/26280 [==============================] - 39s - loss: 0.2335 - acc: 0.9118 - val_loss: 0.0812 - val_acc: 0.9825
Epoch 4/500
26280/26280 [==============================] - 41s - loss: 0.2103 - acc: 0.9213 - val_loss: 0.1306 - val_acc: 0.9582
Epoch 5/500
26280/26280 [==============================] - 39s - loss: 0.1707 - acc: 0.9385 - val_loss: 0.0163 - val_acc: 1.0000
Epoch 6/500
26280/26280 [==============================] - 39s - loss: 0.1783 - acc: 0.9346 - val_loss: 0.1029 - val_acc: 0.9723
Epoch 7/500
26280/26280 [==============================] - 39s - loss: 0.1878 - acc: 0.9282 - val_loss: 0.0182 - val_acc: 1.0000
Epoch 8/500
26280/26280 [=======================

In [14]:
# MLP Model (Except Attention)
model = Sequential()
model.add(Reshape(input_shape=input_shape, target_shape=(25600,)))
model.add(Dense(500, input_shape=input_shape))
model.add(Activation('tanh'))
model.add(Dense(500))
model.add(Activation('tanh'))
model.add(Dense(500))
model.add(Activation('tanh'))
model.add(Dropout(0.25))
model.add(Dense(output_shape))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_4 (Reshape)          (None, 25600)             0         
_________________________________________________________________
dense_7 (Dense)              (None, 500)               12800500  
_________________________________________________________________
activation_19 (Activation)   (None, 500)               0         
_________________________________________________________________
dense_8 (Dense)              (None, 500)               250500    
_________________________________________________________________
activation_20 (Activation)   (None, 500)               0         
_________________________________________________________________
dense_9 (Dense)              (None, 500)               250500    
_________________________________________________________________
activation_21 (Activation)   (None, 500)               0         
__________

In [15]:
adadelta = Adadelta(lr=1.5, decay=1e-6)
model.compile(optimizer=adadelta,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

In [16]:
# MLP Model (Except Attention)
history = model.fit(x_train, y_train,
                    batch_size=100,
                    epochs=500,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 26280 samples, validate on 2921 samples
Epoch 1/500
26280/26280 [==============================] - 44s - loss: 0.8398 - acc: 0.5497 - val_loss: 0.6816 - val_acc: 0.6303
Epoch 2/500
26280/26280 [==============================] - 42s - loss: 0.7568 - acc: 0.5595 - val_loss: 0.6818 - val_acc: 0.6217
Epoch 3/500
26280/26280 [==============================] - 42s - loss: 0.7261 - acc: 0.5697 - val_loss: 0.6535 - val_acc: 0.6303
Epoch 4/500
26280/26280 [==============================] - 41s - loss: 0.7016 - acc: 0.5847 - val_loss: 0.6560 - val_acc: 0.6303
Epoch 5/500
26280/26280 [==============================] - 42s - loss: 0.6854 - acc: 0.5947 - val_loss: 0.6523 - val_acc: 0.6303
Epoch 6/500
26280/26280 [==============================] - 42s - loss: 0.6735 - acc: 0.6036 - val_loss: 0.6613 - val_acc: 0.6303
Epoch 7/500
26280/26280 [==============================] - 42s - loss: 0.6684 - acc: 0.6153 - val_loss: 0.6492 - val_acc: 0.6303
Epoch 8/500
26280/26280 [=======================

In [20]:
# MLP Model (Including Attention)
amodel = Sequential()
amodel.add(Reshape(input_shape=input_shape, target_shape=(25600,)))

attention_model = Sequential()
attention_model.add(amodel)
attention_model.add((Activation('softmax', name='Attention_softmax')))
model = Sequential()
model.add(Merge([amodel, attention_model], mode='mul'))




model.add(Dense(500, input_shape=input_shape))
model.add(Activation('tanh'))
model.add(Dense(500))
model.add(Activation('tanh'))
model.add(Dense(500))
model.add(Activation('tanh'))
model.add(Dropout(0.25))
model.add(Dense(output_shape))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_3 (Merge)              (None, 25600)             0         
_________________________________________________________________
dense_12 (Dense)             (None, 500)               12800500  
_________________________________________________________________
activation_24 (Activation)   (None, 500)               0         
_________________________________________________________________
dense_13 (Dense)             (None, 500)               250500    
_________________________________________________________________
activation_25 (Activation)   (None, 500)               0         
_________________________________________________________________
dense_14 (Dense)             (None, 500)               250500    
_________________________________________________________________
activation_26 (Activation)   (None, 500)               0         
__________

C:\Program Files\Anaconda3\envs\venv\lib\site-packages\ipykernel\__main__.py:9: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.


In [21]:
# MLP Model (Including Attention)
adadelta = Adadelta(lr=1.5, decay=1e-6)
model.compile(optimizer=adadelta,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=100,
                    epochs=500,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 26280 samples, validate on 2921 samples
Epoch 1/500
26280/26280 [==============================] - 45s - loss: 0.6447 - acc: 0.6430 - val_loss: 0.5901 - val_acc: 0.6861
Epoch 2/500
26280/26280 [==============================] - 42s - loss: 0.5711 - acc: 0.7083 - val_loss: 0.5618 - val_acc: 0.7080
Epoch 3/500
26280/26280 [==============================] - 43s - loss: 0.5435 - acc: 0.7311 - val_loss: 0.5632 - val_acc: 0.7087
Epoch 4/500
26280/26280 [==============================] - 44s - loss: 0.5270 - acc: 0.7416 - val_loss: 0.5530 - val_acc: 0.7121
Epoch 5/500
26280/26280 [==============================] - 44s - loss: 0.5110 - acc: 0.7498 - val_loss: 0.5425 - val_acc: 0.7172
Epoch 6/500
26280/26280 [==============================] - 44s - loss: 0.4962 - acc: 0.7584 - val_loss: 0.5449 - val_acc: 0.7206
Epoch 7/500
26280/26280 [==============================] - 42s - loss: 0.4859 - acc: 0.7668 - val_loss: 0.5384 - val_acc: 0.7128
Epoch 8/500
26280/26280 [=======================

In [22]:
# 2D-CNN Model (Including Attention)
amodel = Sequential()
amodel.add(Reshape(input_shape=input_shape, target_shape=(64, 8, 50)))

attention_model = Sequential()
attention_model.add(amodel)
attention_model.add((Activation('softmax', name='Attention_softmax')))
model = Sequential()
model.add(Merge([amodel, attention_model], mode='mul'))

model.add(Convolution2D(64, 3, 3, border_mode='valid'))
model.add(Activation('tanh'))
model.add(Convolution2D(64, 3, 3, border_mode='valid'))
model.add(Activation('tanh'))
model.add(Convolution2D(64, 3, 3, border_mode='valid'))
model.add(Activation('tanh'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Activation('tanh'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(output_shape))
model.add(Activation('softmax'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_4 (Merge)              (None, 64, 8, 50)         0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 62, 6, 64)         28864     
_________________________________________________________________
activation_28 (Activation)   (None, 62, 6, 64)         0         
_________________________________________________________________
conv2d_5 (Conv2D)            (None, 60, 4, 64)         36928     
_________________________________________________________________
activation_29 (Activation)   (None, 60, 4, 64)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 58, 2, 64)         36928     
_________________________________________________________________
activation_30 (Activation)   (None, 58, 2, 64)         0         
__________

C:\Program Files\Anaconda3\envs\venv\lib\site-packages\ipykernel\__main__.py:9: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
C:\Program Files\Anaconda3\envs\venv\lib\site-packages\ipykernel\__main__.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="valid")`
C:\Program Files\Anaconda3\envs\venv\lib\site-packages\ipykernel\__main__.py:13: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="valid")`
C:\Program Files\Anaconda3\envs\venv\lib\site-packages\ipykernel\__main__.py:15: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, (3, 3), padding="valid")`


In [23]:
# 2D-CNN Model (Including Attention)

adadelta = Adadelta(lr=1.5, decay=1e-6)
model.compile(optimizer=adadelta,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=100,
                    epochs=500,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 26280 samples, validate on 2921 samples
Epoch 1/500
26280/26280 [==============================] - 54s - loss: 0.1408 - acc: 0.9506 - val_loss: 2.3089e-04 - val_acc: 1.0000
Epoch 2/500
26280/26280 [==============================] - 45s - loss: 2.4337e-04 - acc: 1.0000 - val_loss: 1.0691e-05 - val_acc: 1.0000
Epoch 3/500
26280/26280 [==============================] - 47s - loss: 3.4950e-05 - acc: 1.0000 - val_loss: 4.0582e-06 - val_acc: 1.0000
Epoch 4/500
26280/26280 [==============================] - 51s - loss: 1.9344e-05 - acc: 1.0000 - val_loss: 2.1722e-06 - val_acc: 1.0000
Epoch 5/500
26280/26280 [==============================] - 49s - loss: 1.1772e-05 - acc: 1.0000 - val_loss: 1.3982e-06 - val_acc: 1.0000
Epoch 6/500
26280/26280 [==============================] - 49s - loss: 8.2419e-06 - acc: 1.0000 - val_loss: 1.1078e-06 - val_acc: 1.0000
Epoch 7/500
26280/26280 [==============================] - 52s - loss: 6.6047e-06 - acc: 1.0000 - val_loss: 8.8134e-07 - val_acc: 1.0

KeyboardInterrupt: 

In [24]:
# 3D-CNN Model (Including Attention)
amodel = Sequential()
amodel.add(Reshape(input_shape=input_shape, target_shape=(8, 8, 8, 50)))


attention_model = Sequential()
attention_model.add(amodel)
attention_model.add((Activation('softmax', name='Attention_softmax')))
model = Sequential()
model.add(Merge([amodel, attention_model], mode='mul'))

model.add(Convolution3D(64, 3, 3, 3, border_mode='valid'))
model.add(Activation('tanh'))
model.add(Convolution3D(64, 3, 3, 3, border_mode='valid'))
model.add(Activation('tanh'))
model.add(Convolution3D(64, 3, 3, 3, border_mode='valid'))
model.add(Activation('tanh'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Activation('tanh'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(output_shape))
model.add(Activation('softmax'))

model.summary()

C:\Program Files\Anaconda3\envs\venv\lib\site-packages\ipykernel\__main__.py:10: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
C:\Program Files\Anaconda3\envs\venv\lib\site-packages\ipykernel\__main__.py:12: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), padding="valid")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_5 (Merge)              (None, 8, 8, 8, 50)       0         
_________________________________________________________________
conv3d_10 (Conv3D)           (None, 6, 6, 6, 64)       86464     
_________________________________________________________________
activation_34 (Activation)   (None, 6, 6, 6, 64)       0         
_________________________________________________________________
conv3d_11 (Conv3D)           (None, 4, 4, 4, 64)       110656    
_________________________________________________________________
activation_35 (Activation)   (None, 4, 4, 4, 64)       0         
_________________________________________________________________
conv3d_12 (Conv3D)           (None, 2, 2, 2, 64)       110656    
_________________________________________________________________
activation_36 (Activation)   (None, 2, 2, 2, 64)       0         
__________

C:\Program Files\Anaconda3\envs\venv\lib\site-packages\ipykernel\__main__.py:14: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), padding="valid")`
C:\Program Files\Anaconda3\envs\venv\lib\site-packages\ipykernel\__main__.py:16: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), padding="valid")`


In [25]:
# 3D-CNN Model (Including Attention)

adadelta = Adadelta(lr=1.5, decay=1e-6)
model.compile(optimizer=adadelta,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=100,
                    epochs=500,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 26280 samples, validate on 2921 samples
Epoch 1/500
26280/26280 [==============================] - 64s - loss: 0.0709 - acc: 0.9704 - val_loss: 1.7985e-04 - val_acc: 1.0000
Epoch 2/500
26280/26280 [==============================] - 57s - loss: 1.5171e-04 - acc: 1.0000 - val_loss: 2.3993e-05 - val_acc: 1.0000
Epoch 3/500
26280/26280 [==============================] - 56s - loss: 3.1612e-05 - acc: 1.0000 - val_loss: 1.6391e-05 - val_acc: 1.0000
Epoch 4/500
 3500/26280 [==>...........................] - ETA: 46s - loss: 1.9554e-05 - acc: 1.0000

KeyboardInterrupt: 

In [26]:
# 3D-LRCN Model (Including Attention)
amodel = Sequential()
amodel.add(Reshape(input_shape=input_shape, target_shape=(50, 8, 8, 8, 1)))

# Attention
attention_model = Sequential()
attention_model.add(amodel)
attention_model.add((Activation('softmax', name='Attention_softmax')))
model = Sequential()
model.add(Merge([amodel, attention_model], mode='mul'))

# 3D-CNN
model.add(TimeDistributed(Convolution3D(64, 3, 3, 3, border_mode='valid', activation='tanh')))
model.add(TimeDistributed(Convolution3D(128, 3, 3, 3, border_mode='valid', activation='tanh')))
model.add(TimeDistributed(Convolution3D(256, 3, 3, 3, border_mode='valid', activation='tanh')))
model.add(TimeDistributed(MaxPooling3D(pool_size=(2, 2, 2))))
model.add(TimeDistributed(Activation('tanh')))
model.add(TimeDistributed(Dropout(0.25)))
model.add(TimeDistributed(Flatten()))


# LSTM
model.add(LSTM(256, return_sequences=False))
model.add(Dense(output_shape))
model.add(Activation('sigmoid'))

model.summary()

C:\Program Files\Anaconda3\envs\venv\lib\site-packages\ipykernel\__main__.py:11: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
C:\Program Files\Anaconda3\envs\venv\lib\site-packages\ipykernel\__main__.py:14: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), padding="valid", activation="tanh")`
C:\Program Files\Anaconda3\envs\venv\lib\site-packages\ipykernel\__main__.py:15: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (3, 3, 3), padding="valid", activation="tanh")`
C:\Program Files\Anaconda3\envs\venv\lib\site-packages\ipykernel\__main__.py:16: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(256, (3, 3, 3), padding="valid", activation="tanh")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
merge_6 (Merge)              (None, 50, 8, 8, 8, 1)    0         
_________________________________________________________________
time_distributed_8 (TimeDist (None, 50, 6, 6, 6, 64)   1792      
_________________________________________________________________
time_distributed_9 (TimeDist (None, 50, 4, 4, 4, 128)  221312    
_________________________________________________________________
time_distributed_10 (TimeDis (None, 50, 2, 2, 2, 256)  884992    
_________________________________________________________________
time_distributed_11 (TimeDis (None, 50, 1, 1, 1, 256)  0         
_________________________________________________________________
time_distributed_12 (TimeDis (None, 50, 1, 1, 1, 256)  0         
_________________________________________________________________
time_distributed_13 (TimeDis (None, 50, 1, 1, 1, 256)  0         
__________

In [27]:
# 3D-LRCN Model (Including Attention)

adadelta = Adadelta(lr=1.5, decay=1e-6)
model.compile(optimizer=adadelta,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=100,
                    epochs=500,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 26280 samples, validate on 2921 samples
Epoch 1/500
26280/26280 [==============================] - 338s - loss: 0.5910 - acc: 0.6791 - val_loss: 0.5421 - val_acc: 0.7850
Epoch 2/500
26280/26280 [==============================] - 330s - loss: 0.3415 - acc: 0.8361 - val_loss: 0.0620 - val_acc: 0.9890
Epoch 3/500
26280/26280 [==============================] - 319s - loss: 0.0614 - acc: 0.9745 - val_loss: 1.6445e-04 - val_acc: 1.0000
Epoch 4/500
26280/26280 [==============================] - 320s - loss: 0.0964 - acc: 0.9639 - val_loss: 0.2525 - val_acc: 0.9089
Epoch 5/500
26280/26280 [==============================] - 323s - loss: 0.0164 - acc: 0.9938 - val_loss: 6.3124e-05 - val_acc: 1.0000
Epoch 6/500
26280/26280 [==============================] - 309s - loss: 0.1504 - acc: 0.9408 - val_loss: 0.4895 - val_acc: 0.6618
Epoch 7/500
26280/26280 [==============================] - 310s - loss: 0.0208 - acc: 0.9918 - val_loss: 5.3069e-05 - val_acc: 1.0000
Epoch 8/500
26280/26280 [====

KeyboardInterrupt: 

In [28]:
# 3D-LRCN Model (Except Attention)
model = Sequential()
model.add(Reshape(input_shape=input_shape, target_shape=(50, 8, 8, 8, 1)))

# 3D-CNN
model.add(TimeDistributed(Convolution3D(64, 3, 3, 3, border_mode='valid', activation='tanh')))
model.add(TimeDistributed(Convolution3D(128, 3, 3, 3, border_mode='valid', activation='tanh')))
model.add(TimeDistributed(Convolution3D(256, 3, 3, 3, border_mode='valid', activation='tanh')))
model.add(TimeDistributed(MaxPooling3D(pool_size=(2, 2, 2))))
model.add(TimeDistributed(Activation('tanh')))
model.add(TimeDistributed(Dropout(0.25)))
model.add(TimeDistributed(Flatten()))


# LSTM
model.add(LSTM(256, return_sequences=False))
model.add(Dense(output_shape))
model.add(Activation('sigmoid'))

model.summary()

C:\Program Files\Anaconda3\envs\venv\lib\site-packages\ipykernel\__main__.py:7: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), padding="valid", activation="tanh")`
C:\Program Files\Anaconda3\envs\venv\lib\site-packages\ipykernel\__main__.py:8: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(128, (3, 3, 3), padding="valid", activation="tanh")`
C:\Program Files\Anaconda3\envs\venv\lib\site-packages\ipykernel\__main__.py:9: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(256, (3, 3, 3), padding="valid", activation="tanh")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_10 (Reshape)         (None, 50, 8, 8, 8, 1)    0         
_________________________________________________________________
time_distributed_15 (TimeDis (None, 50, 6, 6, 6, 64)   1792      
_________________________________________________________________
time_distributed_16 (TimeDis (None, 50, 4, 4, 4, 128)  221312    
_________________________________________________________________
time_distributed_17 (TimeDis (None, 50, 2, 2, 2, 256)  884992    
_________________________________________________________________
time_distributed_18 (TimeDis (None, 50, 1, 1, 1, 256)  0         
_________________________________________________________________
time_distributed_19 (TimeDis (None, 50, 1, 1, 1, 256)  0         
_________________________________________________________________
time_distributed_20 (TimeDis (None, 50, 1, 1, 1, 256)  0         
__________

In [30]:
# 3D-LRCN Model (Except Attention)

adadelta = Adadelta(lr=1.5, decay=1e-6)
model.compile(optimizer=adadelta,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=100,
                    epochs=500,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 26280 samples, validate on 2921 samples
Epoch 1/500
26280/26280 [==============================] - 310s - loss: 0.3380 - acc: 0.8454 - val_loss: 0.1515 - val_acc: 0.9466
Epoch 2/500
26280/26280 [==============================] - 309s - loss: 0.0422 - acc: 0.9850 - val_loss: 0.4403 - val_acc: 0.8456
Epoch 3/500
26280/26280 [==============================] - 310s - loss: 0.0165 - acc: 0.9936 - val_loss: 6.7227e-05 - val_acc: 1.0000
Epoch 4/500
26280/26280 [==============================] - 309s - loss: 0.1052 - acc: 0.9591 - val_loss: 2.1637e-04 - val_acc: 1.0000
Epoch 5/500
26280/26280 [==============================] - 309s - loss: 0.0726 - acc: 0.9726 - val_loss: 1.7359e-04 - val_acc: 1.0000
Epoch 6/500
26280/26280 [==============================] - 309s - loss: 0.0243 - acc: 0.9946 - val_loss: 0.0152 - val_acc: 0.9969
Epoch 7/500
26280/26280 [==============================] - 309s - loss: 0.0013 - acc: 0.9996 - val_loss: 3.9543e-05 - val_acc: 1.0000
Epoch 8/500
26280/26280 [

KeyboardInterrupt: 

In [33]:
# 3D-CNN Model (Except Attention)
model = Sequential()
model.add(Reshape(input_shape=input_shape, target_shape=(8, 8, 8, 50)))


model.add(Convolution3D(64, 3, 3, 3, border_mode='valid'))
model.add(Activation('tanh'))
model.add(Convolution3D(64, 3, 3, 3, border_mode='valid'))
model.add(Activation('tanh'))
model.add(Convolution3D(64, 3, 3, 3, border_mode='valid'))
model.add(Activation('tanh'))
model.add(MaxPooling3D(pool_size=(2, 2, 2)))
model.add(Activation('tanh'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('tanh'))
model.add(Dropout(0.5))
model.add(Dense(output_shape))
model.add(Activation('sigmoid'))

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
reshape_13 (Reshape)         (None, 8, 8, 8, 50)       0         
_________________________________________________________________
conv3d_22 (Conv3D)           (None, 6, 6, 6, 64)       86464     
_________________________________________________________________
activation_51 (Activation)   (None, 6, 6, 6, 64)       0         
_________________________________________________________________
conv3d_23 (Conv3D)           (None, 4, 4, 4, 64)       110656    
_________________________________________________________________
activation_52 (Activation)   (None, 4, 4, 4, 64)       0         
_________________________________________________________________
conv3d_24 (Conv3D)           (None, 2, 2, 2, 64)       110656    
_________________________________________________________________
activation_53 (Activation)   (None, 2, 2, 2, 64)       0         
__________

C:\Program Files\Anaconda3\envs\venv\lib\site-packages\ipykernel\__main__.py:6: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), padding="valid")`
C:\Program Files\Anaconda3\envs\venv\lib\site-packages\ipykernel\__main__.py:8: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), padding="valid")`
C:\Program Files\Anaconda3\envs\venv\lib\site-packages\ipykernel\__main__.py:10: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(64, (3, 3, 3), padding="valid")`


In [ ]:
# 3D-LRCN Model (Except Attention)

adadelta = Adadelta(lr=1.5, decay=1e-6)
model.compile(optimizer=adadelta,
              loss='categorical_crossentropy',
              metrics=['accuracy'])

history = model.fit(x_train, y_train,
                    batch_size=100,
                    epochs=500,
                    verbose=1,
                    validation_data=(x_test, y_test))

Train on 26280 samples, validate on 2921 samples
Epoch 1/500
22400/26280 [========================>.....] - ETA: 6s - loss: 0.6668 - acc: 0.6083